# Building Agents That Use Code

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

## Let's install the dependencies and login to our HF account to access the Inference API

If you haven't installed `smolagents` yet, you can do so by running the following command:

In [ ]:
#!pip install smolagents

Let's also login to the Hugging Face Hub to have access to the Inference API.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## The `@tool` Decorator  

### Generating a tool that retrieves the highest-rated catering

Let's imagine that Alfred has already decided on the menu for the party, but now he needs help preparing food for such a large number of guests. To do so, he would like to hire a catering service and needs to identify the highest-rated options available. Alfred can leverage a tool to search for the best catering services in his area.

Below is an example of how Alfred can use the `@tool` decorator to make this happen:

In [2]:
from smolagents import CodeAgent, HfApiModel, tool, LiteLLMModel
from litellm import completion



# Let's pretend we have a function that fetches the highest-rated catering services.



@tool



def catering_service_tool(query: str) -> str:
    """



    This tool returns the highest-rated catering service in Gotham City.




    Args:



        query: A search term for finding catering services.
    """


    # Example list of catering services and their ratings


    services = {



        "Gotham Catering Co.": 4.9,



        "Wayne Manor Catering": 4.8,



        "Gotham City Events": 4.7,



    }


    # Find the highest rated catering service (simulating search query filtering)


    best_service = max(services, key=services.get)


    return best_service



# Initialize the model using Ollama's local Llama 3.2B model
model = LiteLLMModel(model_id="ollama_chat/llama3.2:3b")

# Create the agent with the Ollama model

agent = CodeAgent(tools=[catering_service_tool], model=model)


# agent = CodeAgent(tools=[catering_service_tool], model=HfApiModel())



# Run the agent to find the best catering service



result = agent.run(



    "Can you give me the name of the highest-rated catering service in Gotham City?"



)



print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me the name of the highest-rated catering service in Gotham City?                                  │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/llama3.2:3b ────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  highest_rated_catering = catering_service_tool(query="Gotham City")                                              
  print(highest_rated_catering)                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Gotham Catering Co.

Out: None

[Step 1: Duration 9.39 seconds| Input tokens: 2,046 | Output tokens: 156]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer('Gotham Catering Co.')                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Gotham Catering Co.

[Step 2: Duration 2.84 seconds| Input tokens: 4,094 | Output tokens: 229]

Gotham Catering Co.


## Defining a Tool as a Python Class  

### Generating a tool to generate ideas about the superhero-themed party

Alfred's party at the mansion is a **superhero-themed event**, but he needs some creative ideas to make it truly special. As a fantastic host, he wants to surprise the guests with a unique theme.

To do this, he can use an agent that generates superhero-themed party ideas based on a given category. This way, Alfred can find the perfect party theme to wow his guests.

In [3]:
from smolagents import Tool, CodeAgent, HfApiModel

class SuperheroPartyThemeTool(Tool):
    name = "superhero_party_theme_generator"
    description = """
    This tool suggests creative superhero-themed party ideas based on a category.
    It returns a unique party theme idea."""

    inputs = {
        "category": {
            "type": "string",
            "description": "The type of superhero party (e.g., 'classic heroes', 'villain masquerade', 'futuristic Gotham').",
        }
    }

    output_type = "string"

    def forward(self, category: str):
        themes = {
            "classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
            "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
            "futuristic Gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets."
        }

        return themes.get(category.lower(), "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic Gotham'.")

# Instantiate the tool
party_theme_tool = SuperheroPartyThemeTool()
#agent = CodeAgent(tools=[party_theme_tool], model=HfApiModel())
agent = CodeAgent(tools=[party_theme_tool], model=model)

# Run the agent to generate a party theme idea
result = agent.run(
    "What would be a good superhero party idea for a 'villain masquerade' theme?"
)

print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What would be a good superhero party idea for a 'villain masquerade' theme?                                     │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/llama3.2:3b ────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\n(.*?)\n``` was not found in it.
            Here is your code snippet:
            A "Villain Masquerade" theme is perfect for a superhero party, as it allows guests to embrace their 
inner villain and have fun playing the role of the bad guys. Here's a party idea that will make your guests feel 
like they're part of a dastardly plot:

**Party Concept:**

Create a mysterious and eerie atmosphere by transforming your party space into a villainous lair. Decorate with 
dark colors, dim lighting, and creepy props like cobwebs, skeletons, and spider webs.

**Invitations:**

Design invitations that look like wanted posters or evil lair announcements. Include the tagline "Join us for a 
night of villainy and debauchery" to set the tone for the party.

**Costumes:**

Encourage guests to come dressed as their favorite supervillains, such as Joker, Lex Luthor, or Magneto. You can 
also provide DIY mask-making stations with materials like feathers, glitter, and cardboard, so guests can create 
their own villainous masks.

**Activities:**

1. **Villainous Games:** Set up a few stations with games that mimic supervillain antics, such as:
 * "Pin the Cackle on the Supervillain" (a spin on Pin the Tail)
 * "The Evil Plan Challenge" (where guests have to assemble a fake evil plan using random props and materials)
 * "Villainous Trivia" (test your guests' knowledge of supervillains and their respective lairs)
2. **Mask-Making Station:** Set up a craft station where guests can create their own villainous masks, just like 
the ones they'll wear to the party.
3. **Superhero Counterattack:** Set up an obstacle course where superheroes (dressed in matching costumes) will try
to stop villains from completing various challenges.
4. **Evil Villain Showdown:** Host a friendly competition where guests can show off their best villainous moves, 
such as breakdancing or doing a rendition of the "Villain's Dance."

**Food and Drinks:**

Serve a menu that fits the theme:

* "Doomsday" cupcakes with evil-themed toppers
* "Evil Eyes" punch (green-colored drink with ice cubes shaped like eyes)
* "Joker's Wild" cookies (shaped like playing cards or jokers)

**Party Favors:**

Give each guest a party favor that fits the theme:

* A DIY mask-making kit
* A packet of villainous-themed stickers or tattoos
* A small evil-themed trinket box filled with candy or treats

This party idea will have your guests laughing, scheming, and having an absolute blast as they channel their inner 
supervillain. So put on the cape, don the mask, and get ready for a night of villainous fun!
            Make sure to include code with the correct pattern, for instance:
            Thoughts: Your thoughts
            Code:
            ```py
            # Your python code here
            ```<end_code>
Make sure to provide correct code blobs.

[Step 1: Duration 11.60 seconds| Input tokens: 48 | Output tokens: 577]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 15 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Use superhero_party_theme_generator tool                                                                       
  party_theme = superhero_party_theme_generator(category='classic heroes')                                         
                                                                                                                   
  # Print the party theme                                                                                          
  print("Superhero Party Theme:", party_theme)                                                                     
                                                                                                                   
  # Use final_answer to provide a final answer                                                                     
  final_answer(party_theme)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Superhero Party Theme: Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails 
like 'The Kryptonite Punch'.

Out - Final answer: Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like
'The Kryptonite Punch'.

[Step 15: Duration 7.36 seconds| Input tokens: 28,720 | Output tokens: 2,966]

Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.


## Sharing a Tool to the Hub

Sharing your custom tool with the community is easy! Simply upload it to your Hugging Face account using the `push_to_hub()` method.

For instance, Alfred can share his `catering_service_tool` to help others find the best catering services in Gotham. Here's how to do it:

In [ ]:
party_theme_tool.push_to_hub("{your_username}/catering_service_tool", token="<YOUR_HUGGINGFACEHUB_API_TOKEN>")

## Importing a Tool from the Hub

You can easily import tools created by other users using the `load_tool()` function. For example, Alfred might want to generate a promotional image for the party using AI. Instead of building a tool from scratch, he can leverage a predefined one from the community:

In [6]:
from smolagents import load_tool, CodeAgent, HfApiModel

image_generation_tool = load_tool(
    "m-ric/text-to-image",
    trust_remote_code=True
)

agent = CodeAgent(
    tools=[image_generation_tool],
    model=model, # HfApiModel(),
    additional_authorized_imports=["m-ric/text-to-image"],
)

#agent.run("Generate an image of a luxurious superhero-themed party at Wayne Manor with made-up superheros.")
agent.run(
    "Generate an image of a snow capped mountain with grazing happy cows on the lush meadows below."
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Generate an image of a snow capped mountain with grazing happy cows on the lush meadows below.                  │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/llama3.2:3b ────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\n(.*?)\n``` was not found in it.
            Here is your code snippet:
            I'm not capable of directly generating images, but I can provide you with a written description and 
some suggestions on how to create an image based on this description.

Here's a written description of the scene:

"Imagine a majestic snow-capped mountain standing tall in the background, its peak shrouded in mist. In the 
foreground, a lush meadow stretches out as far as the eye can see, dotted with happy cows grazing peacefully under 
the warm sun. The cows' coats glisten with dew, and their ears perk up at every sound, as if listening to the 
gentle rustling of the grass.

The sky above is a brilliant blue, with just a few wispy clouds scattered across it. A faint layer of mist lingers 
around the mountain's base, giving the scene an ethereal quality. The trees in the distance are sparse and gnarled,
their branches etched against the vibrant green meadow like nature's own brushstrokes.

The cows themselves are grazing happily, their heads bent low to the ground as they munch on the sweet grass. Their
faces are relaxed
            Make sure to include code with the correct pattern, for instance:
            Thoughts: Your thoughts
            Code:
            ```py
            # Your python code here
            ```<end_code>
Make sure to provide correct code blobs.

[Step 1: Duration 5.54 seconds| Input tokens: 47 | Output tokens: 219]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 7 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # define the prompt for the image                                                                                
  prompt = "a high-res, photorealistic picture of a futuristic cityscape at sunset"                                
                                                                                                                   
  # use the final_answer tool to generate text based on the prompt                                                 
  answer = final_answer({'question': prompt})                                                                      
                                                                                                                   
  # use the answer as input to the image_generator tool                                                            
  image = image_generator(answer={'description': answer})                                                          
                                                                                                                   
  try:                                                                                                             
      # try to execute the code                                                                                    
      image = image_generator(answer={'description': final_answer({'question': "a high-res, photorealistic         
  picture of a futuristic cityscape at sunset"})})                                                                 
  except Exception as e:                                                                                           
      print(f"An error occurred: {e}")                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: {'question': 'a high-res, photorealistic picture of a futuristic cityscape at sunset'}

[Step 7: Duration 5.87 seconds| Input tokens: 12,335 | Output tokens: 1,570]

{'question': 'a high-res, photorealistic picture of a futuristic cityscape at sunset'}

## Importing a Hugging Face Space as a Tool

You can also import a HF Space as a tool using `Tool.from_space()`. This opens up possibilities for integrating with thousands of spaces from the community for tasks from image generation to data analysis.

The tool will connect with the spaces Gradio backend using the `gradio_client`, so make sure to install it via `pip` if you don't have it already. For the party, Alfred can also use a HF Space directly for the generation of the previous annoucement AI-generated image. Let's build it!

In [7]:
%pip install gradio_client

Note: you may need to restart the kernel to use updated packages.


In [8]:
from smolagents import CodeAgent, HfApiModel, Tool

image_generation_tool = Tool.from_space(
    "black-forest-labs/FLUX.1-schnell",
    name="image_generator",
    description="Generate an image from a prompt"
)

#model = HfApiModel("Qwen/Qwen2.5-Coder-32B-Instruct")

agent = CodeAgent(tools=[image_generation_tool], model=model)

agent.run(
    "Improve this prompt, then generate an image of it.",
    additional_args={'user_prompt': 'A grand superhero-themed party at Wayne Manor, with Alfred overseeing a luxurious gala'}
)

Loaded as API: https://black-forest-labs-flux-1-schnell.hf.space ✔


Since `api_name` was not defined, it was automatically set to the first available API: `/infer`.


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Improve this prompt, then generate an image of it.                                                              │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'user_prompt': 'A grand superhero-themed party at Wayne Manor, with Alfred overseeing a luxurious gala'}.      │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/llama3.2:3b ────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\n(.*?)\n``` was not found in it.
            Here is your code snippet:
            Here is an improved version of the prompt:

"Design a lavish and opulent superhero-themed party at Wayne Manor, where guests are treated to an evening of 
luxury and celebration. The esteemed butler, Alfred, ensures that every detail is meticulously attended to, from 
the exquisite decorations to the delectable cuisine."

I made several changes to improve the clarity and specificity of the prompt:

* Added more descriptive language to paint a vivid picture in the mind's eye
* Specified the setting (Wayne Manor) to help narrow down the scope of the design
* Emphasized Alfred's role as butler to provide context for his character
* Suggested specific elements that could be included in the party, such as decorations and cuisine

Here is an image based on this improved prompt:
            Make sure to include code with the correct pattern, for instance:
            Thoughts: Your thoughts
            Code:
            ```py
            # Your python code here
            ```<end_code>
Make sure to provide correct code blobs.

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 10 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Unknown")                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Unknown

[Step 10: Duration 2.29 seconds| Input tokens: 18,502 | Output tokens: 1,996]

'Unknown'

In [ ]:
from PIL import Image as PILImage
import matplotlib.pyplot as plt

image_path = '/tmp/gradio/d5a8dfbade97e9b9d99f78d5ccaa73db6d4b8dc428662f2f25bde1de1bd77b81/image.webp'

img = PILImage.open(image_path)
img

## Importing a LangChain Tool

These tools need a [SerpApi API Key](https://serpapi.com/).

You can easily load LangChain tools using the `Tool.from_langchain()` method. Alfred, ever the perfectionist, is preparing for a spectacular superhero night at Wayne Manor while the Waynes are away. To make sure every detail exceeds expectations, he taps into LangChain tools to find top-tier entertainment ideas.

By using `Tool.from_langchain()`, Alfred effortlessly adds advanced search functionalities to his smolagent, enabling him to discover exclusive party ideas and services with just a few commands.

Here's how he does it:

In [9]:
%pip install langchain-community google-search-results

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 18.1 MB/s eta 0:00:00
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32171 sha256=c2422232f1e0a9a89ec77a8fd87ab60eebb3525ed415c12000d018cc841cf82c
  Stored in directory: c:\users\susanne\appdata\local\pip\cache\wheels\6e\42\3e\aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results
Not

In [ ]:
from google.colab import userdata
import os
os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

In [ ]:
from langchain.agents import load_tools
from smolagents import CodeAgent, HfApiModel, Tool

search_tool = Tool.from_langchain(load_tools(["serpapi"])[0])

agent = CodeAgent(tools=[search_tool], model=model)

agent.run("Search for luxury entertainment ideas for a superhero-themed event, such as live performances and interactive experiences.")

With this setup, Alfred can quickly discover luxurious entertainment options, ensuring Gotham's elite guests have an unforgettable experience. This tool helps him curate the perfect superhero-themed event for Wayne Manor! 🎉